# Jax-Dataloader

> 

![Python](https://img.shields.io/pypi/pyversions/jax-relax.svg)
![CI status](https://github.com/BirkhoffG/ReLax/actions/workflows/test.yaml/badge.svg)
![Docs](https://github.com/BirkhoffG/ReLax/actions/workflows/deploy.yaml/badge.svg)
![pypi](https://img.shields.io/pypi/v/jax-relax)
![GitHub License](https://img.shields.io/github/license/BirkhoffG/ReLax)


This file will become your README and also the index of your documentation.

## Install

```sh
pip install jax_dataloader
```